In [1]:
! pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 2.4 MB/s eta 0:00:00 0:00:01
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.18
    Uninstalling urllib3-1.26.18:
      Successfully uninstalled urllib3-1.26.18


In [7]:
import os

os.environ["OPENAI_API_KEY"]=""
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGHCAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = ''

In [4]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter # split the text?
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma # connect to chromadb?
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [8]:
# Load -> chunk and index contents

loader = WebBaseLoader(
    web_paths=('https://lilianweng.github.io/posts/2023-06-23-agent/',),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=('post-content', 'post-title', 'post-header')
        )
    ),
)

docs = loader.load()

#split and embed
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [9]:
print(vectorstore)

In [10]:
# retrieve and generate 

retriever = vectorstore.as_retriever()

prompt = hub.pull('rlm/rag-prompt')
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.4)

def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()} 
    | prompt
    | llm
    | StrOutputParser()
)

In [11]:
rag_chain.invoke("What is task decomposition")

'Task decomposition is a technique that breaks down complex tasks into smaller and simpler steps to enhance model performance. It involves transforming big tasks into multiple manageable tasks by thinking step by step. This process can be done using simple prompts, task-specific instructions, or with human inputs.'

In [12]:
a = 5
test_chain = (
    a = a+5
    | print("hello")
)

SyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (3078711016.py, line 3)

In [16]:
a = 5
test_chain = (
    a == 5
    | print("Checking chains")
)

test_chain.invoke()

Checking chains


TypeError: unsupported operand type(s) for |: 'int' and 'NoneType'